<a href="https://colab.research.google.com/github/mafemunoz/Trabajo-IA-2025-2/blob/main/04%20-%20modelo%20con%20preprocesado%20por%20moda%20y%20catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Carga el archivo kaggle.json
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!python -m pip install -qq kaggle
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia

Saving kaggle (2).json to kaggle (2).json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
unzip:  cannot find or open udea-ai-4-eng-20252-pruebas-saber-pro-colombia, udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip or udea-ai-4-eng-20252-pruebas-saber-pro-colombia.ZIP.


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.3 MB/s eta 0:00:00


In [5]:
test_df = pd.read_csv('/content/test.csv', encoding= 'utf-8')
df = pd.read_csv('/content/train.csv', encoding= 'utf-8')

El preprocesamiento implementado es el mismo que el modelo 03, manteniendo la limpieza de la columna PROGRAMA como en versiones anteriores. En este caso, las columnas categóricas y numéricas con valores faltantes fueron completadas utilizando la moda de cada una. Además, se eliminaron columnas irrelevantes y duplicadas como E_PRIVADO_LIBERTAD y la repetida F_TIENEINTERNET.1. Para facilitar su manipulación, se renombraron las columnas, se convirtieron los valores booleanos a binarios, y se aplicó codificación One-Hot a todas las variables categóricas seleccionadas. Finalmente, la variable RENDIMIENTO_GLOBAL fue transformada a un valor ordinal, permitiendo su uso en modelos supervisados que requieren variables numéricas.

In [6]:
def Preprocesar(df):
  def obtener_primera_frase(cadena):
    return cadena.split(' ', 1)[0]

  def quitar_tildes(palabra):
    return re.sub(r'[ÁÉÍÓÚ]', lambda x: {'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}[x.group()], palabra)

  df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(obtener_primera_frase)
  df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(quitar_tildes)

  reemplazos = {
      'ADMINISTRACION': ['ADMINISTRACION', 'ADMINISTRACI¿N', 'ADMINISTRACIÒN', 'ADMINISTRACIÓN', 'ADMINSITRACION', 'ADMINSITRACIÓN'],
      'ANTROPOLOGIA': ['ANTROPOLOGIA', 'ANTROPOLOGÍA'],
      'ARTES': ['ARTE', 'ARTES','BELLAS','ANIMACION'],
      'BIOINGENIERIA': ['BIOINGENIERIA', 'BIOINGENIERÍA'],
      'BIOLOGIA': ['BIOLOGIA', 'BIOLOGÍA'],
      'COMUNICACION': ['COMUNICACION', 'COMUNICACIONES', 'COMUNICACI¿N', 'COMUNICACIÓN', 'COMUNICACIÖN'],
      'CONSTRUCCION': ['CONSTRUCCION', 'CONSTRUCCIONES'],
      'CONTADURIA': ['CONTADURIA', 'CONTADURÍA'],
      'DISEÑO': ['DISENO', 'DISE¿O', 'DISE¿¿O', 'DISEÑO'],
      'ECOLOGIA': ['ECOLOGIA', 'ECOLOGÍA'],
      'ECONOMIA': ['ECONOM¿A'],
      'GEOLOGIA': ['GEOLOG¿A'],
      'GERENCIA': ['GERONTOLOGIA'],
      'INGENIERIA': ['INGENIER¿A', 'INGENIERÌA', 'INGENIIERIA','AGROINDUSTRIA'],
      'INSTRUMENTACION': ['INTRUMENTACION'],
      'MATEMATICAS': ['MATEMATICA'],
      'MARKETING': ['MERCADEO', 'MERCADOLOGIA'],
      'PSICOLOGIA': ['PSICOLOGA'],
      'QUIMICA': ['QUIMICA.', 'QU¿MICA'],
      'MUSICA': ['MUSICA-'],
      'TERAPIAS': ['TERAPIA', 'TERAPIAS'],
      'GESTION': ['GESTI¿N'],
      'NEGOCIOS' : ['3°'],
      'DEPORTE' : ['ACTIVIDAD'],
      'LICENCIATURA' : ['Licenciatura','LICENC.EN'],
      'LENGUAS' :['LENGUAJES']

  }

  for key, values in reemplazos.items():
      df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].replace(values, key)

  ##Eliminar columnas

  df = df.drop(labels =['E_PRIVADO_LIBERTAD','F_TIENEINTERNET.1', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4'],axis= 1)

  if 'RENDIMIENTO_GLOBAL' in df.columns:
    df.columns = ['ID','PERIODO','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE','RENDIMIENTO_GLOBAL']
  else:
    df.columns = ['ID','PERIODO','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE']

  ##Convertir a binario

  def convertir_num(valor):
      if valor == 'Si':
          return 1
      elif valor == 'No':
          return 0
      else:
          return valor

  # Aplicar la función a todas las columnas del DataFrame
  df = df.applymap(convertir_num)

  ##Imputar Datos

  df['ESTRATO'] = df['ESTRATO'].str.replace('Estrato ', '')

  columnas_binarias = ['TIENE_INTERNET', 'TIENE_LAVADORA', 'TIENE_AUTO',
                        'PAGA_MATRICULA', 'TIENE_COMPUTADOR']

  for columna in columnas_binarias:
      moda = df[columna].mode()[0]
      df[columna].fillna(moda, inplace=True)


  ModaEstrato = df['ESTRATO'].mode()[0]  # Obtener la moda
  df['ESTRATO'].fillna(ModaEstrato, inplace=True)

  ModaPadre = df['EDUCACION_PADRE'].mode()[0]
  df['EDUCACION_PADRE'].fillna(ModaPadre, inplace=True)

  ModaMadre = df['EDUCACION_MADRE'].mode()[0]
  df['EDUCACION_MADRE'].fillna(ModaMadre, inplace=True)

  ValorMatricula = df['VALOR_MATRICULA'].mode()[0]
  df['VALOR_MATRICULA'].fillna(ValorMatricula, inplace=True)

  HorasTrabaja = df['HORAS_TRABAJA'].mode()[0]
  df['HORAS_TRABAJA'].fillna(HorasTrabaja, inplace=True)

  ##Convertir columnas categoricas
  Columnas_categoricas = ["VALOR_MATRICULA","PROGRAMA","DEPARTAMENTO","EDUCACION_MADRE","EDUCACION_PADRE"]

  df[Columnas_categoricas] = df[Columnas_categoricas].astype("category")

  valores_rendimiento = {'alto': 4,'medio-alto': 3,'medio-bajo':2, 'bajo': 1}
  df= df.replace(valores_rendimiento)

  ##Convertir a Onehot
  encoder = OneHotEncoder(sparse_output=False)

  encoded_columns = encoder.fit_transform(df[['HORAS_TRABAJA','PERIODO','VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE']])

  feature_names = encoder.get_feature_names_out(['HORAS_TRABAJA','PERIODO', 'VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE'])

  one_hot_encoded_df = pd.DataFrame(encoded_columns, columns=feature_names)

  df = pd.concat([df, one_hot_encoded_df], axis=1)

  df = df.drop(labels =['HORAS_TRABAJA', 'PERIODO','VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE'],axis= 1)

  return df

In [7]:
df = Preprocesar(df)
test_df = Preprocesar(test_df)

/tmp/ipython-input-3803613976.py:64: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(convertir_num)
/tmp/ipython-input-3803613976.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(moda, inplace=True)
/tmp/ipython-input-3803613976.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the in

In [8]:
print(df.isnull().sum())

ID                                                      0
TIENE_INTERNET                                          0
TIENE_LAVADORA                                          0
TIENE_AUTO                                              0
PAGA_MATRICULA                                          0
                                                       ..
EDUCACION_MADRE_Primaria incompleta                     0
EDUCACION_MADRE_Secundaria (Bachillerato) completa      0
EDUCACION_MADRE_Secundaria (Bachillerato) incompleta    0
EDUCACION_MADRE_Técnica o tecnológica completa          0
EDUCACION_MADRE_Técnica o tecnológica incompleta        0
Length: 199, dtype: int64


**Conclusiones**

Este modelo mostró un desempeño muy favorable desde su implementación inicial. La solución final se desarrolló utilizando este mismo modelo, pero acompañado de un preprocesamiento optimizado, el cual ofreció mejores resultados al momento de realizar las predicciones y subirlas a la plataforma de Kaggle.

In [9]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

X = df.drop(labels =["RENDIMIENTO_GLOBAL",'ID'], axis=1)  # Características
y = df["RENDIMIENTO_GLOBAL"]  # Variable objetivo

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 )

# Crear el modelo CatBoost
catboost_model = CatBoostClassifier(iterations=1500, learning_rate=0.1, depth=6, loss_function='MultiClass')

# Entrenar el modelo
catboost_model.fit(X_train, y_train)

# Evaluar el modelo
accuracy = catboost_model.score(X_test, y_test)
print("Accuracy:", accuracy)

0:	learn: 1.3690886	total: 345ms	remaining: 8m 36s
1:	learn: 1.3561118	total: 598ms	remaining: 7m 27s
2:	learn: 1.3451655	total: 864ms	remaining: 7m 11s
3:	learn: 1.3361808	total: 1.14s	remaining: 7m 8s
4:	learn: 1.3288387	total: 1.39s	remaining: 6m 55s
5:	learn: 1.3223204	total: 1.63s	remaining: 6m 45s
6:	learn: 1.3168388	total: 1.87s	remaining: 6m 38s
7:	learn: 1.3116401	total: 2.16s	remaining: 6m 43s
8:	learn: 1.3071886	total: 2.4s	remaining: 6m 38s
9:	learn: 1.3032630	total: 2.65s	remaining: 6m 35s
10:	learn: 1.3000165	total: 2.9s	remaining: 6m 32s
11:	learn: 1.2968465	total: 3.16s	remaining: 6m 32s
12:	learn: 1.2935124	total: 3.42s	remaining: 6m 30s
13:	learn: 1.2909711	total: 3.65s	remaining: 6m 28s
14:	learn: 1.2886783	total: 3.9s	remaining: 6m 25s
15:	learn: 1.2862716	total: 4.16s	remaining: 6m 25s
16:	learn: 1.2842612	total: 4.43s	remaining: 6m 26s
17:	learn: 1.2822710	total: 4.66s	remaining: 6m 24s
18:	learn: 1.2801243	total: 4.93s	remaining: 6m 24s
19:	learn: 1.2782885	total

In [10]:
predictions = catboost_model.predict(test_df.drop('ID', axis=1))

predictions = predictions.flatten()

submit = pd.DataFrame({'ID': test_df['ID'], 'RENDIMIENTO_GLOBAL': predictions})

submit['RENDIMIENTO_GLOBAL'] = submit['RENDIMIENTO_GLOBAL'].replace({
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto'
})

submit.to_csv('submission.csv', index=False)